# Titanic Survival

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


We are going to study the survival rate of passengers on titanic and what variables affected survival.

Load the dataset in `titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [1]:
from IPython.core.display import HTML
HTML(filename='../data/titanic.html'),

(<IPython.core.display.HTML object>,)

In [2]:
# you would need xlrd - pip install xlrd
import pandas as pd
t_file = pd.ExcelFile('../data/titanic.xls')
t_df = t_file.parse("titanic")
t_df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
t_df.isnull().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

### Keeping columns and dropping NA values

In [4]:
t_df = t_df[['pclass','survived','sex','age']]
t_df.dropna(inplace=True)

### Women and children first?

*** 1. Use the `groupby` method to calculate the proportion of passengers that survived by sex. (10 pts)*** 

### Grouping by sex and survived and using the sex column (no missing values)to calculate proportion of total survived.

### I will also round to the 2nd decimal and append a % sign to the column to indicate that are percentage values

In [47]:
((t_df.groupby(['sex','survived'])['survived'].count()/t_df.groupby(['sex'])['survived'].count()*100).to_frame().rename(columns={'survived':'survived_%_group'})['survived_%_group'].round(2).map(str) + " %").to_frame()

survived_%_group
sex    survived                 
female 0                 24.74 %
       1                 75.26 %
male   0                 79.48 %
       1                 20.52 %

*** 2. Calculate the same proportion, but by class and sex. (10 pts)*** 

### The same thing can be achieved by adding `pclass` to the groupby statement

In [46]:
(((t_df.groupby(['sex',
               'pclass',
               'survived'])['survived'].count()/t_df.groupby(['sex',
                                                              'pclass'])['survived'].count())*100).to_frame().rename(columns={'survived':'survived_%_group'})['survived_%_group'].round(2).map(str) + " %").to_frame()


survived_%_group
sex    pclass survived                 
female 1      0                  3.76 %
              1                 96.24 %
       2      0                 10.68 %
              1                 89.32 %
       3      0                 52.63 %
              1                 47.37 %
male   1      0                  64.9 %
              1                  35.1 %
       2      0                 85.44 %
              1                 14.56 %
       3      0                 83.09 %
              1                 16.91 %

*** 3. Create age categories: children (under 14 years), adolescents (14-20), adult (21-64), and senior(65+), and calculate survival proportions by age category, class and sex. (20 pts)***

### We can create the categories many different ways. I chose to use np.select simply because the code is better read this way. It assigns the category by evaluating a number of statements. The corresponding index of the true statement is matched the the corresponding index of the label in the second list. 

In [30]:
import numpy as np
t_df['age_group'] = np.select([
    t_df['age'].between(0,13, inclusive = True),
    t_df['age'].between(14,20.99, inclusive = True),
    t_df['age'].between(21,64, inclusive = True),
    t_df['age'].between(65,999, inclusive = True),
],
[
    'children',
    'adolescents',
    'adult',
    'senior'
    
],
default = 'unknown_age')

t_df.groupby('age_group')['survived'].count()

age_group
adolescents    150
adult          784
children        99
senior          13
Name: survived, dtype: int64

In [43]:
t_proportion=((t_df.groupby(['survived',
               'sex',
               'pclass',
               'age_group'])['survived'].count()/t_df.groupby(['sex',
                                                               'pclass',
                                                              'age_group'])['survived'].count())*100).to_frame().rename(columns={'survived':'survived_%_group'})['survived_%_group'].round(2).map(str) + " %"

t_proportion.to_frame()

survived_%_group
sex    pclass age_group   survived                 
female 1      adolescents 1                 100.0 %
              adult       0                  3.45 %
                          1                 96.55 %
              children    0                 100.0 %
              senior      1                 100.0 %
       2      adolescents 0                  7.69 %
                          1                 92.31 %
              adult       0                 13.16 %
                          1                 86.84 %
              children    1                 100.0 %
       3      adolescents 0                 45.71 %
                          1                 54.29 %
              adult       0                 55.81 %
                          1                 44.19 %
              children    0                 51.61 %
                          1                 48.39 %
male   1      adolescents 0                  80.0 %
                          1                  20.0 %
              adult       0                 65.67 %
                          1                 34.33 %
              children    1                 100.0 %
              senior      0                 85.71 %
                          1                 14.29 %
       2      adolescents 0                 88.24 %
                          1                 11.76 %
              adult       0                 92.19 %
                          1                  7.81 %
              children    1                 100.0 %
              senior      0                 100.0 %
       3      adolescents 0                 87.69 %
                          1                 12.31 %
              adult       0                 84.02 %
                          1                 15.98 %
              children    0                 67.57 %
                          1                 32.43 %
              senior      0                 100.0 %